In [ ]:
# reference documentation: https://keras.io/examples/nlp/abstractive_summarization_with_bart/

In [2]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras

import os

os.environ["KERAS_BACKEND"] = "tensorflow" #"jax"  # or "tensorflow" or "torch"

import keras_nlp

import keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.1/572.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 38.5 MB/s eta 0:00:00


In [1]:
!pip install -q evaluate
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-c

In [ ]:
!pip install -q rouge_score
!pip install -q bert_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 26.8 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import tensorflow as tf
from datasets import Dataset, load_dataset, load_metric
import nltk
import time
from datetime import datetime, timedelta
import pytz
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/266_Project/envisci_articles.csv')
df.head()

,abstract,full_text
0,Ice nucleation is a process of great relevance...,ordinated water molecules. Ice nucleation fro...
1,The ability of a red tide species to take up n...,INTRODUCTION Red tides or harmful algal bloom...
2,Recent advances in science and technology radi...,RA is a systemic disorder characterized by inf...
3,Secondary ion mass spectrometry (SIMS) is a pr...,Introduction Secondary ion mass spectrometry ...
4,Citizen-science databases have been used to de...,aims in this study were to illustrate the effe...


In [ ]:
df.isna().sum()

abstract     0
full_text    0
dtype: int64

In [ ]:
df = df.reindex(columns=['full_text', 'abstract'])
df.head()

,full_text,abstract
0,ordinated water molecules. Ice nucleation fro...,Ice nucleation is a process of great relevance...
1,INTRODUCTION Red tides or harmful algal bloom...,The ability of a red tide species to take up n...
2,RA is a systemic disorder characterized by inf...,Recent advances in science and technology radi...
3,Introduction Secondary ion mass spectrometry ...,Secondary ion mass spectrometry (SIMS) is a pr...
4,aims in this study were to illustrate the effe...,Citizen-science databases have been used to de...


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=1)

In [ ]:
# Train & Val
train_df, val_df = train_test_split(train, test_size=0.2, random_state=1)

In [4]:
BATCH_SIZE = 1
NUM_BATCHES = 10
EPOCHS = 1
MAX_ENCODER_SEQUENCE_LENGTH = 512
MAX_DECODER_SEQUENCE_LENGTH = 512
MAX_GENERATION_LENGTH = 150
#min_length = int(150/2)
#no_repeat_ngram_size = 4

In [5]:
preprocessor = keras_nlp.models.BartSeq2SeqLMPreprocessor.from_preset(
    "bart_large_en_cnn",
    encoder_sequence_length=MAX_ENCODER_SEQUENCE_LENGTH,
    decoder_sequence_length=MAX_DECODER_SEQUENCE_LENGTH,
)
bart_lm = keras_nlp.models.BartSeq2SeqLM.from_preset(
    "bart_large_en_cnn", preprocessor=preprocessor
)

bart_lm.summary()

100%|██████████| 141/141 [00:00<00:00, 348kB/s]


100%|██████████| 448/448 [00:00<00:00, 977kB/s]


100%|██████████| 0.99M/0.99M [00:00<00:00, 3.01MB/s]


100%|██████████| 446k/446k [00:00<00:00, 1.76MB/s]


100%|██████████| 485/485 [00:00<00:00, 697kB/s]


...

Model: "bart_seq2_seq_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ decoder_padding_mask          │ (None, None)              │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ decoder_token_ids             │ (None, None)              │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ encoder_padding_mask          │ (None, None)              │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ encoder_token_ids             │ (None, None)              │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bart_backbone (BartBackbone)  │ [(None, None, 1024),      │     406,286,336 │ decoder_padding_mask[0][0… │
│                               │ (None, None, 1024)]       │                 │ decoder_token_ids[0][0],   │
│                               │                           │                 │ encoder_padding_mask[0][0… │
│                               │                           │                 │ encoder_token_ids[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50264)       │      51,470,336 │ bart_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 406,286,336 (1.51 GB)

 Trainable params: 406,286,336 (1.51 GB)

 Non-trainable params: 0 (0.00 B)

## Baseline

In [ ]:
len(test)

832

In [ ]:
start_index = 750
index = 832
t = test[start_index:index]

In [ ]:
abstracts = t['abstract'].astype(str).tolist()
full_texts = t['full_text'].astype(str).tolist()

test_ds = tf.data.Dataset.from_tensor_slices((full_texts,abstracts))

test_ds = test_ds.take(100)

In [ ]:
def generate_baseline_text(model, input_text, max_length=512, print_time_taken=False):
    data = []
    start = time.time()
    output = model.generate(input_text, max_length=max_length)
    end = time.time()
    print(f"Total Time Elapsed: {end - start:.2f}s")

    return output

_ = generate_baseline_text(bart_lm, "sample text", max_length=MAX_GENERATION_LENGTH)

generated_baseline_summaries = generate_baseline_text(
    bart_lm,
    test_ds.map(lambda full_text, _: full_text).batch(8),
    max_length=MAX_GENERATION_LENGTH,
    print_time_taken=True,
)

Total Time Elapsed: 10.08s
Total Time Elapsed: 1654.54s


In [ ]:
final = {'GeneratedText':generated_baseline_summaries, 'ReferenceText':abstracts}
df_final = pd.DataFrame(final)
df_final.head()

,GeneratedText,ReferenceText
0,The United Nations Food Systems Summit sought ...,Globalized food systems are a major driver of ...
1,This paper tries to use an interdisciplinary m...,The paper studies the ropeways as a special su...
2,Report: Pfiesteria was first identified and na...,The Pfiesteria organism was first identified a...
3,"Urinary bladder cancer (UBC), is highly ranked...",In some cases a combination of several maligna...
4,"Canada, like many countries, increasingly face...",Canada regularly faces environmental public he...


In [ ]:
# Create Filename
LOCAL_OUTPUT_CSV_PATH = '/content/drive/MyDrive/266_Project/bart_base_summaries/'
file_name = f'bart_baseline_index{index}_maxlen{MAX_GENERATION_LENGTH}.csv'
output_path = LOCAL_OUTPUT_CSV_PATH + file_name

df_final.to_csv(output_path, index=False, header=True)

In [ ]:
for abstract, generated_summary, full_text in zip(
    abstracts[:5], generated_baseline_summaries[:5], full_texts[:5]):
    print("Full Text:", full_text)
    print("Generated Summary:", generated_summary)
    print("abstract:", abstract)
    print("=============================")

In [ ]:
rouge = evaluate.load('rouge')
result = rouge.compute(predictions=generated_baseline_summaries, references=abstracts)

print(result)

## Fine Tuning

In [ ]:
len(train)

3328

In [ ]:
train_df['full_text'] = train_df['full_text'].astype(str)
train_df['abstract'] = train_df['abstract'].astype(str)

train_ds = train_df.rename(
    columns={"full_text": "encoder_text",
             "abstract": "decoder_text"}
    ).to_dict(orient = 'list')

train_ds = tf.data.Dataset.from_tensor_slices((train_ds))

train_ds = train_ds.batch(2).cache() #.batch(5)
#train_ds = next(iter(train_ds.batch(16)))

In [ ]:
len(val_df)

In [ ]:
#v_abstracts = val_df['abstract'].astype(str).tolist()
#v_full_texts = val_df['full_text'].astype(str).tolist()

val_df['full_text'] = val_df['full_text'].astype(str)
val_df['abstract'] = val_df['abstract'].astype(str)
val_ds = val_df.rename(
    columns={"full_text": "encoder_text",
             "abstract": "decoder_text"}
    ).to_dict(orient = 'list')

val_ds = tf.data.Dataset.from_tensor_slices((val_ds))#((v_full_texts,v_abstracts))

val_ds = val_ds.batch(2)

In [ ]:
# save model checkpoints
checkpoint_dir = '/content/drive/MyDrive/266_Project/bart_model_checkpoints/'
checkpoint_file = checkpoint_dir + 'bart_weights.weights.h5'

In [ ]:
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_file,
    save_weights_only=True,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [ ]:
optimizer = keras.optimizers.AdamW(
    learning_rate=.005, # possibly change this
    weight_decay=0.01,
    epsilon=1e-6,
    global_clipnorm=1.0,  # Gradient clipping.
)
# Exclude layernorm and bias terms from weight decay.
optimizer.exclude_from_weight_decay(var_names=["bias"])
optimizer.exclude_from_weight_decay(var_names=["gamma"])
optimizer.exclude_from_weight_decay(var_names=["beta"])

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

bart_lm.compile(
    optimizer=optimizer,
    loss=loss,
    weighted_metrics=["accuracy"],
)

In [ ]:
bart_lm.fit(
    train_ds,
    epochs=2,
    callbacks=[checkpoint_callback],
    validation_data=val_ds,
    shuffle=True,
)

In [ ]:
#bart_lm.save('/content/drive/MyDrive/266_Project/bart_results/bart_results.keras')

In [ ]:
#model = keras.models.load_model('/content/drive/MyDrive/266_Project/bart_results/bart_results.keras')

In [ ]:
test_start_index = 800
test_index = 832
t = test[test_start_index:test_index]

In [ ]:
abstracts = t['abstract'].astype(str).tolist()
full_texts = t['full_text'].astype(str).tolist()

test_ds = tf.data.Dataset.from_tensor_slices((full_texts,abstracts))

#test_ds = test_ds.take(100)

In [ ]:
def generate_text(model, input_text, max_length=512, print_time_taken=False):
    start = time.time()
    output = model.generate(input_text, max_length=max_length)
    end = time.time()
    print(f"Total Time Elapsed: {end - start:.2f}s")
    return output

_ = generate_text(bart_lm, "sample text", max_length=MAX_GENERATION_LENGTH)

Total Time Elapsed: 0.45s


In [ ]:
# Generate summaries.
generated_summaries = generate_text(
    bart_lm,
    test_ds.map(lambda full_text, _: full_text).batch(8),
    max_length=MAX_GENERATION_LENGTH,
    print_time_taken=True,
)

Total Time Elapsed: 28.13s


In [ ]:
final = {'GeneratedText':generated_summaries, 'ReferenceText':abstracts}
df_final = pd.DataFrame(final)
df_final.head()

,GeneratedText,ReferenceText
0,Palaeoichnological research is increasingly be...,The 3D digitisation of palaeontological resour...
1,The importance of sustainability is highlighte...,The comfort of human life depends on the quali...
2,‘Fragilidade ambiental’ das qualidência das ce...,Este artigo aborda o desafio da modelagem da f...
3,The study of 15 iron-rich minerals in an agric...,Iron (oxy)hydroxides play an important role in...
4,The concept of interdisciplinary research proj...,Research projects combining different discipli...


In [ ]:
# Create Filename
LOCAL_OUTPUT_CSV_PATH = '/content/drive/MyDrive/Datasci_266_NLP/266_Project/bart_ft_summaries/'
file_name = f'bart_finetuned_index{test_index}_maxlen{MAX_GENERATION_LENGTH}.csv'
output_path = LOCAL_OUTPUT_CSV_PATH + file_name

df_final.to_csv(output_path, index=False, header=True)

Total Time Elapsed: 62.11s
Total Time Elapsed: 392.80s


In [ ]:
for abstract, generated_summary, ground_truth_summary in zip(
    abstracts[:5], generated_summaries[:5], full_texts[:5]):
    print("Full Text:", ground_truth_summary)
    print("Generated Summary:", generated_summary)
    print("abstract:", abstract)
    print("=============================")

Full Text: INTRODUCTION  The three-dimensional (3D) surface digitisation of both fossil specimens and localities has become a growing trend among palaeontologists over the past decade.Not only has digitisation led to advances in science through the accessibility and flexibility of working with digital models, but it has enabledthe creation of online repositories (e.g., Digimorph -http://www.digimorph.org) for archiving and distributing this data (Smith and Strait, 2008;Belvedere et al., 2011a).   Uses of Digital Specimens  The digitisation of skeletons has enabled researchers to investigate ranges of motion (Chapman et al., 1999;Mallison, 2010aMallison, , 2010b)), constrain soft tissue volumes (Gunga et al., 2008;Bates et al., 2009bBates et al., , 2009d)), and explore aspects of biomechanics including locomotion and feeding (Hutchinson et al., 2005;Rybczynski et al., 2008;Gatesy et al., 2009;Sellers et al., 2009) in extinct animals, free from the limitations of handling large, heavy, a

In [ ]:
rouge = evaluate.load('rouge')
result = rouge.compute(predictions=generated_summaries, references=abstracts)

print('validation set results')
print("=============================")
print(result)

validation set results
{'rouge1': 0.2906187521242565, 'rouge2': 0.06854553445610334, 'rougeL': 0.16399616683328663, 'rougeLsum': 0.16353683849997241}


In [ ]:
# test set on fine tuned model

test_generated_summaries = generate_text(
    bart_lm, #model,
    test_ds.map(lambda full_text, _: full_text).batch(8),
    max_length=MAX_GENERATION_LENGTH,
    print_time_taken=True,
)

Total Time Elapsed: 452.26s


In [ ]:
for abstract, generated_summary, ground_truth_summary in zip(
    abstracts[:5], test_generated_summaries[:5], full_texts[:5]):
    print("Full Text:", ground_truth_summary)
    print("Generated Summary:", generated_summary)
    print("abstract:", abstract)
    print("=============================") # could be interesting to check the number of times the generated summary begins with same word as the full_text

Full Text: INTRODUCTION    Department of Psychiatry Chosun University Hospital GwangjuRepublic of Korea   Department of Psychiatry School of Medicine Department of Psychiatry School of Medicine Chosun University GwangjuRepublic of Korea   Chosun University 365 Pilmun-daero, Dong-gu501-717GwangjuRepublic of Korea  INTRODUCTION 10.4306/pi.2015.12.2.274OPEN ACCESS 274 The prevalence of psychiatric disorders is higher in people with epilepsy than in the general population. In patients with epilepsy, depression occurs in about 30%, anxiety disorders in 10-25%, and psychosis in 2-7%. 1 Thus, these patients are more likely to have psychiatric diseases, 2 and are more than three times as likely to commit suicide as the general population. 2,3 About 5% of the adolescents in the community have Major Depressive Disorder (MDD) and another 3.3% have dysthymia. 3 Evidence showed that MDD is associated with smaller hippocampal volumes. 4-6 A previous study found that subjects with depression and a hi

In [ ]:
rouge = evaluate.load('rouge')
result = rouge.compute(predictions=test_generated_summaries, references=abstracts)

print('test set results')
print("=============================")
print(result)

test set results
{'rouge1': 0.20745587845858182, 'rouge2': 0.15453600737293466, 'rougeL': 0.18068207135332257, 'rougeLsum': 0.17590679495488137}


## Scoring

In [ ]:
rouge = evaluate.load('rouge')
meteor = evaluate.load('meteor')
bertscore = evaluate.load('bertscore')
def calculate_model_rouge_scores(df):
  # count = 1
  est_start = get_current_time_est()

  def compute_rouge_scores(row):
    prediction = row['GeneratedText']
    reference = row['ReferenceText']
    rouge_scores = rouge.compute(predictions=[prediction], references=[reference])
    # count[0] += 1
    # print(f'Example {count[0]} ROUGE scored.')
    return pd.Series(rouge_scores)

  df_rouge = df.apply(compute_rouge_scores, axis=1)
  df = pd.concat([df, df_rouge], axis=1)

  calculate_duration(est_start)

  return df

def calculate_model_meteor_scores(df):
  # count = 1
  est_start = get_current_time_est()


  def compute_meteor_scores(row):
    prediction = row['GeneratedText']
    reference = row['ReferenceText']
    rouge_scores = meteor.compute(predictions=[prediction], references=[reference])
    # count[0] += 1
    # print(f'Example {count[0]} METEOR scored.')
    return pd.Series(rouge_scores)

  df_rouge = df.apply(compute_meteor_scores, axis=1)
  df = pd.concat([df, df_rouge], axis=1)

  calculate_duration(est_start)

  return df

def calculate_model_bert_scores(df):
  est_start = get_current_time_est()

  def compute_bert_score(row):
    prediction = row['GeneratedText']
    reference = row['ReferenceText']
    bert_scores = bertscore.compute(predictions=[prediction], references=[reference], lang="en")
    return pd.Series(bert_scores)

  df_bert = df.apply(compute_bert_score, axis=1)
  df = pd.concat([df, df_bert], axis=1)

  calculate_duration(est_start)

  return df


def get_current_time_est():
  est = pytz.timezone('US/Eastern')
  utc_start = datetime.now(pytz.utc)
  est_start = utc_start.astimezone(est)
  print(f'Start Time: {est_start}')
  return est_start

def calculate_duration(est_start):
  est = pytz.timezone('US/Eastern')
  utc_end = datetime.now(pytz.utc)
  est_end = utc_end.astimezone(est)
  time_difference = est_end - est_start
  seconds = time_difference.seconds
  hours = seconds // 3600
  minutes = (seconds % 3600) // 60
  seconds = seconds % 60
  formatted_difference = f"{hours:02}:{minutes:02}:{seconds:02}"

  print(f'Complete. Time Taken: {formatted_difference}')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# Get a list of all CSV files in a directory
csv_files = glob.glob('/content/drive/MyDrive/Datasci_266_NLP/266_Project/bart_ft_summaries/*.csv')

# Create an empty dataframe to store the combined data
combined_df = pd.DataFrame()

# Loop through each CSV file and append its contents to the combined dataframe
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    combined_df = pd.concat([combined_df, df])

# Print the combined dataframe
print(combined_df)

                                        GeneratedText  \
0   Birdhouse is an integrated web processing tool...   
1   The use of bibliometric mapping to map the stu...   
2   The sustainable development can be viewed as a...   
3   Background: Throughout history, humans have se...   
4   Exposure Ontology (Exposure Ontology) is a new...   
..                                                ...   
27  The concept of environmental justice is well e...   
28  Medical records of 90 patients with drug overd...   
29  We examined the relationship between short-ter...   
30  Geological disasters affect the development of...   
31  The notion of resilience is a global phenomeno...   

                                        ReferenceText  
0   Birdhouse is a collaborative project open for ...  
1   Modern plant phenotyping, often using non-inva...  
2   This paper aims to explore the role of sustain...  
3   Background: Predicting and abating the loss of...  
4   E nvironmental health informati

In [ ]:
combined_df = combined_df.reset_index(drop=True)
combined_df.head()

,GeneratedText,ReferenceText
0,Birdhouse is an integrated web processing tool...,Birdhouse is a collaborative project open for ...
1,The use of bibliometric mapping to map the stu...,"Modern plant phenotyping, often using non-inva..."
2,The sustainable development can be viewed as a...,This paper aims to explore the role of sustain...
3,"Background: Throughout history, humans have se...",Background: Predicting and abating the loss of...
4,Exposure Ontology (Exposure Ontology) is a new...,E nvironmental health information resources la...


In [ ]:
##### Output paths with evaluations
root_evaluation_path = '/content/drive/MyDrive/Datasci_266_NLP/266_Project/model_evaluations/'
file_name = 'bart_finetuned'

In [ ]:
df_bert = calculate_model_bert_scores(combined_df)
df_rouge_bert = calculate_model_rouge_scores(df_bert)
df_meteor_rouge_bert = calculate_model_meteor_scores(df_rouge_bert)
output_path = root_evaluation_path + file_name + '.csv'
df_meteor_rouge_bert.to_csv(output_path, index=False)
print(f'File written to {output_path}.')

Start Time: 2024-07-28 14:01:21.641345-04:00
Complete. Time Taken: 00:01:16
Start Time: 2024-07-28 14:02:38.000609-04:00
Complete. Time Taken: 00:02:42
Start Time: 2024-07-28 14:05:20.320206-04:00
Complete. Time Taken: 00:00:13
File written to /content/drive/MyDrive/Datasci_266_NLP/266_Project/model_evaluations/bart_finetuned.csv.


In [ ]:
df = pd.DataFrame()

for root, dirs, files in os.walk(root_evaluation_path):
  dirs.clear() # do not go down subfolder (e.g. ARCHIVE)
  for file in files:
    if file.endswith('.csv'):
      file_path = os.path.join(root, file)
      model_name = os.path.splitext(file)[0]
      df_model = pd.read_csv(file_path)
      df_model['precision'] = df_model['precision'].str.strip('[]').astype(float)
      df_model['recall'] = df_model['recall'].str.strip('[]').astype(float)
      df_model['f1'] = df_model['f1'].str.strip('[]').astype(float)
      df_model = df_model.select_dtypes(include=['float64'])
      df_model = df_model.mean().to_frame().T
      df_model['ModelName'] = model_name
      df = pd.concat([df, df_model], axis = 0)

df.reset_index(drop=True).head(20)

,precision,recall,f1,rouge1,rouge2,rougeL,rougeLsum,meteor,ModelName
0,0.844748,0.803063,0.822927,0.197573,0.047093,0.132328,0.132328,0.093533,t5_baseline_level_1_150
1,0.844944,0.804371,0.823712,0.200274,0.049058,0.131815,0.131815,0.097350,t5_baseline_level_2_150
2,0.847068,0.803819,0.824513,0.206709,0.048861,0.135043,0.135043,0.098910,t5_baseline_level_3_150
3,0.850295,0.820457,0.834868,0.278465,0.073214,0.166854,0.166854,0.149217,t5_finetune_no_prompt_level_1_150
4,0.850908,0.820199,0.835034,0.280365,0.074281,0.168284,0.168284,0.148704,t5_finetune_no_prompt_level_2_150
5,0.848938,0.819469,0.833721,0.277832,0.071628,0.165061,0.165061,0.149149,t5_finetune_no_prompt_level_3_150
6,0.860174,0.807270,0.832593,0.172510,0.045961,0.114340,0.114340,0.084427,bart_baseline
7,0.850311,0.818923,0.834067,0.282550,0.064702,0.161811,0.161811,0.149373,bart_finetuned
